In [ ]:
from google.colab import drive
import pandas as pd
import math
from collections import Counter
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path_train = '/content/drive/MyDrive/cs5350/hw3/train.csv'
file_path_test = '/content/drive/MyDrive/cs5350/hw3/test.csv'
attributes_train = []
labels_train = []
attributes_test = []
labels_test = []
with open(file_path_train, 'r') as f:
    for line in f:
        terms = line.strip().split(',')
        attr = [float(x) for x in terms[:4]]
        label = int(terms[4])
        attributes_train.append(attr)
        labels_train.append(label)

with open(file_path_test, 'r') as f:
    for line in f:
        terms = line.strip().split(',')
        attr = [float(x) for x in terms[:4]]
        label = int(terms[4])
        attributes_test.append(attr)
        labels_test.append(label)

In [ ]:
import numpy as np
X_train = np.array(attributes_train, dtype=float)
y_train = np.array(labels_train, dtype=int)
X_test = np.array(attributes_test, dtype=float)
y_test = np.array(labels_test, dtype=int)

y_train = np.where(y_train == 0, -1, 1)
y_test = np.where(y_test == 0, -1, 1)

In [ ]:
def standardPerceptron(X_train, y_train, learning_rate=0.1, T=10):
    w = np.zeros(X_train.shape[1] + 1, dtype=float)
    for epoch in range(T):
        indices = np.random.permutation(X_train.shape[0])
        for i in indices:
            x_i = np.insert(X_train[i], 0, 1).astype(float)
            if y_train[i] * np.dot(w, x_i) <= 0:
                w += learning_rate * y_train[i] * x_i
    return w

In [ ]:
def votedPerceptron(X_train, y_train, learning_rate=0.1, T=10):
    w = np.zeros(X_train.shape[1] + 1, dtype=float)
    weight_vectors = []
    counts = []
    m = 0

    for epoch in range(T):
        indices = np.random.permutation(X_train.shape[0])
        for i in indices:
            x_i = np.insert(X_train[i], 0, 1).astype(float)
            if y_train[i] * np.dot(w, x_i) <= 0:
                if m > 0:
                    counts[-1] += 1
                weight_vectors.append(w.copy())
                counts.append(1)
                m += 1
                w += learning_rate * y_train[i] * x_i
            else:
                if counts:
                    counts[-1] += 1
    return weight_vectors, counts


In [ ]:
def predictVotedPerceptron(X, weight_vectors, counts):
    X_with_bias = np.insert(X, 0, 1, axis=1).astype(float)
    predictions = []
    for x in X_with_bias:
        weighted_sum = sum(count * np.sign(np.dot(w, x)) for w, count in zip(weight_vectors, counts))
        predictions.append(np.sign(weighted_sum))
    return np.array(predictions)

In [ ]:
def averagePerceptron(X_train, y_train, learning_rate=0.1, T=10):
    w = np.zeros(X_train.shape[1] + 1, dtype=float)
    a = np.zeros_like(w)

    for epoch in range(T):
        indices = np.random.permutation(X_train.shape[0])
        for i in indices:
            x_i = np.insert(X_train[i], 0, 1).astype(float)
            if y_train[i] * np.dot(w, x_i) <= 0:
                w += learning_rate * y_train[i] * x_i
            a += w
    return a

In [ ]:
from sklearn.metrics import accuracy_score

w_standard = standardPerceptron(X_train, y_train, learning_rate=0.1, T=10)

X_test_with_bias = np.insert(X_test, 0, 1, axis=1).astype(float)
predictions_standard = np.sign(np.dot(X_test_with_bias, w_standard))
accuracy_standard = accuracy_score(y_test, np.where(predictions_standard == -1, 0, 1))

average_error_standard = 1 - accuracy_standard
print("Standard learned weight vector:", w_standard[1:])
print("Average prediction error Standard Perceptron:", average_error_standard)

Standard learned weight vector: [-6.030707  -3.843418  -3.6166041 -1.1041335]
Average prediction error Standard Perceptron: 0.5660000000000001


In [ ]:
weight_vectors, counts = votedPerceptron(X_train, y_train, learning_rate=0.1, T=10)

print("Voted weight vectors and counts:")
for i, (w, c) in enumerate(zip(weight_vectors, counts)):
    print(f"Weight vector {i}: {w}, Count: {c}")

Voted weight vectors and counts:
Weight vector 0: [0. 0. 0. 0. 0.], Count: 2
Weight vector 1: [ 0.1      -0.36012  -0.65389   1.05234  -0.048967], Count: 3
Weight vector 2: [ 0.2      -0.54227  -0.37868   0.980079 -0.284267], Count: 3
Weight vector 3: [ 0.3      -0.452758  0.0987    0.495769 -0.843357], Count: 2
Weight vector 4: [ 0.2      -0.436076 -0.49104   0.44593  -0.773313], Count: 11
Weight vector 5: [ 0.1      -0.366504 -1.35269   0.26174  -0.340423], Count: 4
Weight vector 6: [ 2.77555756e-17 -6.54204000e-01 -9.46700000e-01 -1.00850000e-01
 -3.07879000e-01], Count: 17
Weight vector 7: [ 0.1      -0.503434 -0.75074  -0.40669  -0.320122], Count: 12
Weight vector 8: [ 2.77555756e-17 -9.68434000e-01 -2.67770000e-01 -7.52220000e-01
 -2.94948000e-01], Count: 6
Weight vector 9: [-0.1      -0.993469 -1.20039  -0.38349   0.330482], Count: 14
Weight vector 10: [ 2.77555756e-17 -9.37237000e-01 -1.10024000e+00 -6.10750000e-01
  3.29877140e-01], Count: 4
Weight vector 11: [ 0.1        -1.1

In [ ]:
predictions_voted = predictVotedPerceptron(X_test, weight_vectors, counts)
accuracy_voted = accuracy_score(y_test, np.where(predictions_voted == -1, 0, 1))

average_error_voted = 1 - accuracy_voted
print("Average prediction error Voted Perceptron:", average_error_voted)

Average prediction error Voted Perceptron: 0.5660000000000001


In [ ]:
w_average = averagePerceptron(X_train, y_train, learning_rate=0.1, T=10)

predictions_average = np.sign(np.dot(X_test_with_bias, w_average))
accuracy_average = accuracy_score(y_test, np.where(predictions_average == -1, 0, 1))

average_error_average = 1 - accuracy_average
print("Average learned weight vector:", w_average[1:])
print("Average prediction error Average Perceptron:", average_error_average)

Average learned weight vector: [-41926.9927537 -25914.283657  -26762.4355355  -7394.6339682]
Average prediction error Average Perceptron: 0.5640000000000001


2c) When comparing the list of weight vectors from voted perceptron to the average perceptron learned weight vector, they are extremely different and even magnitudes apart. It is possible that there was a lot of noise or variablity in the data.

2d) When comparing the three perceptron methods, the predicted errors were very similar. The average has the lowest and standard has the highest. This is probably due to the fact standard relys on one weight vector and average takes the average weight.